In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/KalmanNet_Visual/KalmanNet_VO/

/content/drive/MyDrive/KalmanNet_Visual/KalmanNet_VO


In [ ]:
import os
import torch
import sys
from datetime import datetime
from Extended_sysmdl_visual import SystemModel
import numpy as np
from EKF_test_visual import EKFTest
from model import f, h_full, h_partial, h_nonlinear
from encoder_ae_models import Encoder_new as Encoder

import matplotlib.pyplot as plt
import copy

if torch.cuda.is_available():
    dev = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    dev = torch.device("cpu")
    print("Running on the CPU")
ROOT_PATH_DRIVE = r"/content/drive/MyDrive/KalmanNet_Visual/KalmanNet_VO/"
ROOT_PATH_LOCAL = r"C:\Users\damis\KalmanNetDrive\KalmanNet_Visual\KalmanNet_VO/"
ROOT_PATH = ROOT_PATH_DRIVE

Running on the CPU
Running on the CPU
Running on the CPU
Running on the CPU


In [ ]:
####################################################################
##### EXPERIMENT KEEPING Q CONSTANT, ENCODER + EKF ##
####################################################################


opt_wrt_to = "theta"
NL_m1_0 = torch.FloatTensor([0.5, 0])
NL_m2_0 = torch.tensor([[0.0, 0.0], [0.0, 0.0]])
NL_T = 399 #sequence length for training/cv set, here we not used
NL_T_test = 399 #sequence length for test set
random_init = 0
true_init = 1
probs = [1.0, 1-1/32, 1-1/16, 1-1/8, 1-1/4, 1/2, 1/4, 1/8,  1/16, 1/32]
#probs = [0.5]
r2s = [10, 2, 1, 0.5, 0.1, 0.01, 0.001, 0.0001]
q2s = [0, -10, -20]
q2s = [0]
q2s = list(map(lambda x: 10**(x/10), q2s))
v = 0
modelKnow = "partial"


print(f"OBSERVATION MODEL: {modelKnow}")
for q2 in q2s:
  target_test_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")["test_set"]
  input_ = np.load(ROOT_PATH+f"Datasets/Pendulum/images_clean/pendulum_images_clean_q2_{q2:.0e}_v_{v}.npz")
  target_val_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")["validation_set"]
  for p in probs:
    print(f"Evaluation for q2 {q2}, p {p} \n")
    model_dir = ROOT_PATH+f"Simulations_results/Pendulum/Encoder_big_net_optimal_hyp_big/salt&pepper/models/encoder_p_{p}.pt"
    loss_dir = ROOT_PATH+f"Simulations_results/Pendulum/Encoder_big_net_optimal_hyp_big/salt&pepper/losses/loss_p_{p}.npz"

    net = Encoder(encoded_dimension=1)
    print("Load encoder model")
    net.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
    

    r2 = np.load(loss_dir)["MSE_avg_linear_rad"]

    q2_grid = [q2, 10*q2, 100*q2, 15*q2, 0.1*q2, 0.01*q2, 20*q2, 50*q2, 1.1*q2, 1.2*q2, 1.3*q2, 0.9*q2, 0.8*q2, 0.7*q2, 0.6*q2]
    #q2_grid = [q2]
    #r2_grid = [r2, 10*r2, 100*r2, 15*r2, 0.1*r2, 0.01*r2, 20*r2, 50*r2, 1.1*r2, 1.2*r2, 1.3*r2, 0.9*r2, 0.8*r2, 0.7*r2, 0.6*r2]
    r2_grid = [r2]
    best_q2 = q2
    best_r2 = r2
    best_MSE = 1000
    r = np.sqrt(r2)






    if modelKnow == "partial":
      print("Generate noisy images")
      clean_test = torch.from_numpy(input_["test_set"].copy()).to(dev)
      clean_val = torch.from_numpy(input_["validation_set"][:200, ...].copy()).to(dev)
      probabilities_test = torch.ones_like(clean_test) * p
      probabilities_val = torch.ones_like(clean_val) * p
      locations_test = torch.bernoulli(probabilities_test)
      locations_val = torch.bernoulli(probabilities_val)
      random_array_test = torch.randint(0, 255, locations_test.shape)
      random_array_val = torch.randint(0, 255, locations_val.shape)
      noisy_test = clean_test * (torch.ones_like(locations_test) - locations_test) + locations_test * random_array_test
      noisy_val = clean_val * (torch.ones_like(locations_val) - locations_val) + locations_val * random_array_val
      normalized_test = torch.clamp(noisy_test, min=0, max=255) / 255
      normalized_val = torch.clamp(noisy_val, min=0, max=255) / 255


      target_test = torch.from_numpy(target_test_).float().to(dev)
      target_val = torch.from_numpy(target_val_[:200, ...]).float().to(dev)
      h = h_partial
      NL_m = 2  # DIMENSION OF STATE
      NL_n = 1

    input_test = normalized_test.float().to(dev)
    input_val = normalized_val.float().to(dev)

    

    print(f"Start grid search on validation set")
    for q2_g in q2_grid:
      for r2_g in r2_grid:
        print(f"Evaluate grid-value q2: {q2_g} ")
        r = np.sqrt(r2_g)
        q = np.sqrt(q2_g)
        sys_model = SystemModel(f, q, h, r, NL_T, NL_T_test, NL_m, NL_n, "pendulum")
        sys_model.InitSequence(NL_m1_0, NL_m2_0)

        [MSE_EKF_linear_arr_total, MSE_EKF_linear_arr_theta, MSE_EKF_linear_arr_omega, MSE_EKF_dB_avg_total, MSE_EKF_dB_avg_theta,
          MSE_EKF_dB_avg_omega, MSE_EKF_dB_std_total, MSE_EKF_dB_std_theta, MSE_EKF_dB_std_omega,
          KG_array, EKF_out] = EKFTest(SysModel=sys_model, test_input=input_val,
                                                            test_target=target_val, model_AE_conv=net,
                                                            matrix_data_flag=False, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)

        print(f"best_mSE: {best_MSE}, MSE_total: {MSE_EKF_dB_avg_total}, MSE_"+opt_wrt_to+":"+str(eval("MSE_EKF_dB_avg_"+opt_wrt_to)))
        if eval("MSE_EKF_dB_avg_"+opt_wrt_to) < best_MSE:
          best_MSE = eval("MSE_EKF_dB_avg_"+opt_wrt_to).item()
          best_q2 = q2_g
          best_r2 = r2_g

        print("\n")
    
    print(f"Evaluation on test set q_2, r2: {q2}, {r2}, with q2_best: {best_q2}, and r2_best: {best_r2}")


    sys_model = SystemModel(f, np.sqrt(best_q2), h, np.sqrt(best_r2), NL_T, NL_T_test, NL_m, NL_n, "pendulum")
    sys_model.InitSequence(NL_m1_0, NL_m2_0)

    [MSE_EKF_linear_arr_total, MSE_EKF_linear_arr_theta, MSE_EKF_linear_arr_omega, MSE_EKF_dB_avg_total, MSE_EKF_dB_avg_theta,
    MSE_EKF_dB_avg_omega, MSE_EKF_dB_std_total, MSE_EKF_dB_std_theta, MSE_EKF_dB_std_omega,
    KG_array, EKF_out] = EKFTest(SysModel=sys_model, test_input=input_test, test_target=target_test,
                                model_AE_conv=net, matrix_data_flag=False, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)
    """
    sys_model = SystemModel(f, np.sqrt(q2), h, np.sqrt(r2), NL_T, NL_T_test, NL_m, NL_n, "pendulum")
    sys_model.InitSequence(NL_m1_0, NL_m2_0)
    
    [_, _, _, MSE_EKF_dB_avg_total_original, MSE_EKF_dB_avg_theta_original, MSE_EKF_dB_avg_omega_original, _, _, _, _, _] = \
      EKFTest(SysModel=sys_model, test_input=input_test, test_target=target_test, model_AE_conv=net,
              matrix_data_flag=False, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)
    print(f"SAVING MSE_DB_TOTAL TEST SET : {MSE_EKF_dB_avg_total}")
    """
    os.makedirs(ROOT_PATH+f"Simulations_results/Pendulum/EKF+Encoder/constant_q2_experiment_encoder_q2_grid/{modelKnow}_observation_model/", exist_ok=True)

    np.savez(ROOT_PATH+f"Simulations_results/Pendulum/EKF+Encoder/constant_q2_experiment_encoder_q2_grid/{modelKnow}_observation_model/const_q2_{q2:.0e}_p_{p}.npz",
            MSE_linear_arr_total=MSE_EKF_linear_arr_total.cpu(), MSE_linear_arr_theta=MSE_EKF_linear_arr_theta.cpu(),
            MSE_linear_arr_omega=MSE_EKF_linear_arr_omega.cpu(), MSE_db_avg_total=MSE_EKF_dB_avg_total.cpu(),
            MSE_db_avg_theta=MSE_EKF_dB_avg_theta.cpu(), MSE_db_avg_omega=MSE_EKF_dB_avg_omega.cpu(),
            MSE_EKF_dB_std_total=MSE_EKF_dB_std_total.cpu(), MSE_EKF_dB_std_theta=MSE_EKF_dB_std_theta.cpu(),
            MSE_EKF_dB_std_omega=MSE_EKF_dB_std_omega.cpu(), KG_array=KG_array.cpu(), results_plain=EKF_out.cpu(),
            best_q2=best_q2, r2=best_r2)
    print("\n")


OBSERVATION MODEL: partial
Evaluation for q2 1.0, p 1.0 

Load encoder model
Generate noisy images
Start grid search on validation set
Evaluate grid-value q2: 1.0 
EKF - MSE LOSS TOTAL: tensor(-1.0988) [dB]
EKF - MSE STD TOTAL: tensor(2.6519) [dB]
Inference Time: 59.033507108688354
best_mSE: 1000, MSE_total: -1.0987987518310547, MSE_theta:tensor(-7.3592)


Evaluate grid-value q2: 10.0 
EKF - MSE LOSS TOTAL: tensor(2.1678) [dB]
EKF - MSE STD TOTAL: tensor(2.4715) [dB]
Inference Time: 58.5397207736969
best_mSE: -7.359233379364014, MSE_total: 2.167785167694092, MSE_theta:tensor(-4.2845)


Evaluate grid-value q2: 100.0 
EKF - MSE LOSS TOTAL: tensor(4.3083) [dB]
EKF - MSE STD TOTAL: tensor(2.4065) [dB]
Inference Time: 58.21608543395996
best_mSE: -7.359233379364014, MSE_total: 4.308279037475586, MSE_theta:tensor(-3.4038)


Evaluate grid-value q2: 15.0 
EKF - MSE LOSS TOTAL: tensor(2.5870) [dB]
EKF - MSE STD TOTAL: tensor(2.4535) [dB]
Inference Time: 58.62845158576965
best_mSE: -7.35923337936

In [ ]:
####################################################################
##### EXPERIMENT KEEPING Q CONSTANT, CHANGE R AND CONSEQUENTLY NU ##
####################################################################

#modelKnow = "partial"
    # DIMENSION of observation
opt_wrt_to = "theta"
NL_m1_0 = torch.FloatTensor([0.5, 0])
NL_m2_0 = torch.tensor([[0.0, 0.0], [0.0, 0.0]])
NL_T = 399 #sequence length for training/cv set, here we not used 
NL_T_test = 399 #sequence length for test set
random_init = 0
true_init = 1


r2s = [10, 2, 1, 0.5, 0.1, 0.01, 0.001, 0.0001]
q2s = [0, -10, -20]
q2s = list(map(lambda x: 10**(x/10), q2s))
v=0

for modelKnow in ["partial"]:
  print(f"OBSERVATION MODEL: {modelKnow}")
  for q2 in q2s:
    input_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")
    target_test_ = np.copy(input_["test_set"])
    target_val_ = np.copy(input_["validation_set"][:200, ...])
    for r2 in r2s:
      print(f"Evaluation for r2 {r2}, q2 {q2} \n")
      #q2_grid = [q2, 10*q2, 100*q2, 15*q2, 0.1*q2, 0.01*q2, 20*q2, 50*q2, 1.1*q2, 1.2*q2, 1.3*q2, 0.9*q2, 0.8*q2, 0.7*q2, 0.6*q2]
      q2_grid = [q2]
      r2_grid = [r2]
      best_q2 = q2
      best_r2 = r2
      best_MSE = 1000
      r = np.sqrt(r2)


      if modelKnow == "full":
        input_test = torch.from_numpy(input_["test_set"][:, :, :]) + torch.randn(input_["test_set"][:, :, :].shape) * r
        target_test = torch.from_numpy(target_test_).float().to(dev)

        input_val = torch.from_numpy(input_["validation_set"][:200, :, :]) + torch.randn(input_["validation_set"][:200, :, :].shape) * r
        target_val = torch.from_numpy(target_val_).float().to(dev)
        h = h_full
        NL_m = 2  # DIMENSION OF STATE
        NL_n = 2

      if modelKnow == "nonlinear":
        input_test = input_["test_set"]
        target_test = torch.from_numpy(target_test_).float().to(dev)

        input_test = input_test[:, 0:1, :]
        input_test = np.sin(input_test) + np.random.randn(*input_test.shape) * r
        input_test = torch.from_numpy(input_test).float().to(dev)

        input_val = input_["validation_set"][:200, 0:1, ...]
        target_val = torch.from_numpy(target_val_).float().to(dev)

        input_val = np.sin(input_val) + np.random.randn(*input_val.shape) * r
        input_val = torch.from_numpy(input_val).float().to(dev)
        h = h_nonlinear
        NL_m = 2  # DIMENSION OF STATE
        NL_n = 1


      if modelKnow == "partial":
        input_test = torch.from_numpy(input_["test_set"][:, 0:1, :]) + torch.randn(input_["test_set"][:, 0:1, :].shape,) * r
        target_test = torch.from_numpy(target_test_).float().to(dev)
        input_val = torch.from_numpy(input_["validation_set"][:200, 0:1, :]) + torch.randn(input_["validation_set"][:200, 0:1, :].shape) * r
        target_val = torch.from_numpy(target_val_).float().to(dev)
        h = h_partial
        NL_m = 2  # DIMENSION OF STATE
        NL_n = 1

      input_test = input_test.float().to(dev)
      input_val = input_val.float().to(dev)

      
      """
      print(f"Start grid search on validation set")
      for q2_g in q2_grid:
        for r2_g in r2_grid:
          print(f"Evaluate grid-value q2: {q2_g} ")
          r = np.sqrt(r2_g)
          q = np.sqrt(q2_g)
          sys_model = SystemModel(f, q, h, r, NL_T, NL_T_test, NL_m, NL_n, "pendulum")
          sys_model.InitSequence(NL_m1_0, NL_m2_0)

          [MSE_EKF_linear_arr_total, MSE_EKF_linear_arr_theta, MSE_EKF_linear_arr_omega, MSE_EKF_dB_avg_total, MSE_EKF_dB_avg_theta,
            MSE_EKF_dB_avg_omega, MSE_EKF_dB_std_total, MSE_EKF_dB_std_theta, MSE_EKF_dB_std_omega,
            KG_array, EKF_out] = EKFTest(SysModel=sys_model, test_input=input_val,
                                                              test_target=target_val, model_AE_conv=None,
                                                              matrix_data_flag=True, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)

          print(f"best_mSE: {best_MSE}, MSE_total: {MSE_EKF_dB_avg_total}, MSE_"+opt_wrt_to+":"+str(eval("MSE_EKF_dB_avg_"+opt_wrt_to)))
          if eval("MSE_EKF_dB_avg_"+opt_wrt_to) < best_MSE:
            best_MSE = eval("MSE_EKF_dB_avg_"+opt_wrt_to).item()
            best_q2 = q2_g
            best_r2 = r2_g

          print("\n")
      """
      print(f"Evaluation on test set q_2, r2: {q2}, {r2}, with q2_best: {best_q2}, and r2_best: {best_r2}")


      sys_model = SystemModel(f, np.sqrt(best_q2), h, np.sqrt(best_r2), NL_T, NL_T_test, NL_m, NL_n, "pendulum")
      sys_model.InitSequence(NL_m1_0, NL_m2_0)

      [MSE_EKF_linear_arr_total, MSE_EKF_linear_arr_theta, MSE_EKF_linear_arr_omega, MSE_EKF_dB_avg_total, MSE_EKF_dB_avg_theta,
      MSE_EKF_dB_avg_omega, MSE_EKF_dB_std_total, MSE_EKF_dB_std_theta, MSE_EKF_dB_std_omega,
      KG_array, EKF_out] = EKFTest(SysModel=sys_model, test_input=input_test, test_target=target_test,
                                  model_AE_conv=None, matrix_data_flag=True, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)
      sys_model = SystemModel(f, np.sqrt(q2), h, np.sqrt(r2), NL_T, NL_T_test, NL_m, NL_n, "pendulum")
      sys_model.InitSequence(NL_m1_0, NL_m2_0)
      """
      [_, _, _, MSE_EKF_dB_avg_total_original, MSE_EKF_dB_avg_theta_original, MSE_EKF_dB_avg_omega_original, _, _, _, _, _] = \
        EKFTest(SysModel=sys_model, test_input=input_test, test_target=target_test, model_AE_conv=None,
                matrix_data_flag=True, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)
      print(f"SAVING MSE_DB_TOTAL TEST SET : {MSE_EKF_dB_avg_total}")
      """
      os.makedirs(ROOT_PATH+f"Simulations_results/Pendulum/EKF/constant_q2_experiment_NO_GRIDD/{modelKnow}_observation_model_opt_wrt_{opt_wrt_to}/", exist_ok=True)
      np.savez(ROOT_PATH+f"Simulations_results/Pendulum/EKF/constant_q2_experiment_NO_GRIDD/{modelKnow}_observation_model_opt_wrt_{opt_wrt_to}/const_q2_{q2:.0e}_r2_{r2:.0e}.npz",
              MSE_linear_arr_total=MSE_EKF_linear_arr_total.cpu(), MSE_linear_arr_theta=MSE_EKF_linear_arr_theta.cpu(),
              MSE_linear_arr_omega=MSE_EKF_linear_arr_omega.cpu(), MSE_db_avg_total=MSE_EKF_dB_avg_total.cpu(),
              MSE_db_avg_theta=MSE_EKF_dB_avg_theta.cpu(), MSE_db_avg_omega=MSE_EKF_dB_avg_omega.cpu(),
              MSE_EKF_dB_std_total=MSE_EKF_dB_std_total.cpu(), MSE_EKF_dB_std_theta=MSE_EKF_dB_std_theta.cpu(),
              MSE_EKF_dB_std_omega=MSE_EKF_dB_std_omega.cpu(), KG_array=KG_array.cpu(), results_plain=EKF_out.cpu(),
              best_q2=best_q2, best_r2=best_r2)
      print("\n")
      """
      np.savez(ROOT_PATH+f"Simulations_results/Pendulum/EKF/constant_q2_experiment_new/{modelKnow}_observation_model_opt_wrt_{opt_wrt_to}/const_q2_{q2:.0e}_r2_{r2:.0e}.npz",
              MSE_linear_arr_total=MSE_EKF_linear_arr_total.cpu(), MSE_linear_arr_theta=MSE_EKF_linear_arr_theta.cpu(),
              MSE_linear_arr_omega=MSE_EKF_linear_arr_omega.cpu(), MSE_db_avg_total=MSE_EKF_dB_avg_total.cpu(),
              MSE_db_avg_theta=MSE_EKF_dB_avg_theta.cpu(), MSE_db_avg_omega=MSE_EKF_dB_avg_omega.cpu(),
              MSE_EKF_dB_std_total=MSE_EKF_dB_std_total.cpu(), MSE_EKF_dB_std_theta=MSE_EKF_dB_std_theta.cpu(),
              MSE_EKF_dB_std_omega=MSE_EKF_dB_std_omega.cpu(), KG_array=KG_array.cpu(), results_plain=EKF_out.cpu(),
              best_q2=best_q2, best_r2=best_r2, MSE_original_total=MSE_EKF_dB_avg_total_original.cpu(),
              MSE_original_theta=MSE_EKF_dB_avg_theta_original.cpu(), MSE_original_omega=MSE_EKF_dB_avg_omega_original.cpu())
      print("\n")
      """

OBSERVATION MODEL: partial
Evaluation for r2 10, q2 1.0 

Evaluation on test set q_2, r2: 1.0, 10, with q2_best: 1.0, and r2_best: 10
EKF - MSE LOSS TOTAL: tensor(-6.5183) [dB]
EKF - MSE STD TOTAL: tensor(2.4620) [dB]
Inference Time: 76.45654034614563


Evaluation for r2 2, q2 1.0 

Evaluation on test set q_2, r2: 1.0, 2, with q2_best: 1.0, and r2_best: 2
EKF - MSE LOSS TOTAL: tensor(-7.6667) [dB]
EKF - MSE STD TOTAL: tensor(1.9142) [dB]
Inference Time: 75.3165590763092


Evaluation for r2 1, q2 1.0 

Evaluation on test set q_2, r2: 1.0, 1, with q2_best: 1.0, and r2_best: 1
EKF - MSE LOSS TOTAL: tensor(-8.3105) [dB]
EKF - MSE STD TOTAL: tensor(1.8424) [dB]
Inference Time: 74.84612321853638


Evaluation for r2 0.5, q2 1.0 

Evaluation on test set q_2, r2: 1.0, 0.5, with q2_best: 1.0, and r2_best: 0.5
EKF - MSE LOSS TOTAL: tensor(-8.9645) [dB]
EKF - MSE STD TOTAL: tensor(1.7188) [dB]
Inference Time: 74.58402633666992


Evaluation for r2 0.1, q2 1.0 

Evaluation on test set q_2, r2: 1.0, 

In [ ]:
##################################################################
##### EXPERIMENT KEEPING NU CONSTANT; q,r change accordingly #####
##################################################################


#modelKnow = "partial"
opt_wrt_to = "total"
NL_m1_0 = torch.FloatTensor([0.5, 0])
NL_m2_0 = torch.tensor([[0.0, 0.0], [0.0, 0.0]])
NL_T = 399 #sequence length for training/cv set, here we not used
NL_T_test = 399 #sequence length for test set
random_init = 0
true_init = 1

vs = [0,-10,-20] #in [dB]
r2s = [10, 2, 1, 0.5, 0.1, 0.01, 0.001, 0.0001]



for modelKnow in ["partial", "full", "nonlinear"]:
  for v in vs:
    q2s = list(map(lambda x: x*(10**(v/10)), r2s))
    for r2, q2 in zip(r2s, q2s):
        print(f"Evaluation for r2 {r2}, q2 {q2} \n")
        q2_grid = [q2, 10*q2, 100*q2, 15*q2, 0.1*q2, 0.01*q2, 20*q2, 50*q2, 1.1*q2, 1.2*q2, 1.3*q2, 0.9*q2, 0.8*q2, 0.7*q2, 0.6*q2]
        #q2_grid = [q2]
        r2_grid = [r2]
        best_q2 = q2
        best_r2 = r2
        best_MSE = 1000
        r = np.sqrt(r2)


        if modelKnow == "full":
          input_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_noisy_data/pendulum_decimated_noisy_q2_{q2:.0e}_r2_{r2:.0e}_v{v}.npz")
          target = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")
          input_test = torch.from_numpy(input_["test_set"][:, :, :]).float().to(dev)
          target_test = torch.from_numpy(target["test_set"]).float().to(dev)
          input_val = torch.from_numpy(input_["validation_set"][:200, :, :]).float().to(dev)
          target_val = torch.from_numpy(target["validation_set"][:200, ...]).float().to(dev)
          h = h_full
          NL_m = 2  # DIMENSION OF STATE
          NL_n = 2

        if modelKnow == "nonlinear":
          input = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")
          input_test = input["test_set"]
          target_test = np.copy(input_test)
          target_test = torch.from_numpy(target_test).float().to(dev)

          input_test = input_test[:, 0:1, :]
          input_test = np.sin(input_test) + np.random.randn(*input_test.shape) * r
          input_test = torch.from_numpy(input_test).float().to(dev)

          input_val = input["validation_set"][:200, ...]
          target_val = np.copy(input_val)
          target_val = torch.from_numpy(target_val).float().to(dev)

          input_val = input_val[:, 0:1, :]
          input_val = np.sin(input_val) + np.random.randn(*input_val.shape) * r
          input_val = torch.from_numpy(input_val).float().to(dev)
          h = h_nonlinear
          NL_m = 2  # DIMENSION OF STATE
          NL_n = 1


        if modelKnow == "partial":
          input_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_noisy_data/pendulum_decimated_noisy_q2_{q2:.0e}_r2_{r2:.0e}_v{v}.npz")
          target = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")
          input_test = torch.from_numpy(input_["test_set"][:, 0:1, :]).float().to(dev)
          target_test = torch.from_numpy(target["test_set"]).float().to(dev)
          input_val = torch.from_numpy(input_["validation_set"][:200, 0:1, :]).float().to(dev)
          target_val = torch.from_numpy(target["validation_set"][:200, ...]).float().to(dev)
          h = h_partial
          NL_m = 2  # DIMENSION OF STATE
          NL_n = 1

        
        print(f"Start grid search on validation set")
        for q2_g in q2_grid:
          for r2_g in r2_grid:
            print(f"Evaluate grid-value q2: {q2_g} ")
            r = np.sqrt(r2_g)
            q = np.sqrt(q2_g)
            sys_model = SystemModel(f, q, h, r, NL_T, NL_T_test, NL_m, NL_n, "pendulum")
            sys_model.InitSequence(NL_m1_0, NL_m2_0)

            [MSE_EKF_linear_arr_total, MSE_EKF_linear_arr_theta, MSE_EKF_linear_arr_omega, MSE_EKF_dB_avg_total, MSE_EKF_dB_avg_theta,
              MSE_EKF_dB_avg_omega, MSE_EKF_dB_std_total, MSE_EKF_dB_std_theta, MSE_EKF_dB_std_omega,
              KG_array, EKF_out] = EKFTest(SysModel=sys_model, test_input=input_val,
                                                                test_target=target_val, model_AE_conv=None,
                                                                matrix_data_flag=True, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)

            print(f"best_mSE: {best_MSE}, MSE_total: {MSE_EKF_dB_avg_total}, MSE_"+opt_wrt_to+":"+str(eval("MSE_EKF_dB_avg_"+opt_wrt_to)))
            if eval("MSE_EKF_dB_avg_"+opt_wrt_to) < best_MSE:
                best_MSE = eval("MSE_EKF_dB_avg_"+opt_wrt_to).item()
                best_q2 = q2_g
                best_r2 = r2_g

            print("\n")
        

        print(f"Evaluation on test set q_2, r2: {q2}, {r2}, with q2_best: {best_q2}, and r2_best: {best_r2}")


        sys_model = SystemModel(f, np.sqrt(best_q2), h, np.sqrt(best_r2), NL_T, NL_T_test, NL_m, NL_n, "pendulum")
        sys_model.InitSequence(NL_m1_0, NL_m2_0)

        [MSE_EKF_linear_arr_total, MSE_EKF_linear_arr_theta, MSE_EKF_linear_arr_omega, MSE_EKF_dB_avg_total, MSE_EKF_dB_avg_theta,
          MSE_EKF_dB_avg_omega, MSE_EKF_dB_std_total, MSE_EKF_dB_std_theta, MSE_EKF_dB_std_omega,
          KG_array, EKF_out] = EKFTest(SysModel=sys_model, test_input=input_test, test_target=target_test,
                                      model_AE_conv=None, matrix_data_flag=True, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)
        """
        sys_model = SystemModel(f, np.sqrt(q2), h, np.sqrt(r2), NL_T, NL_T_test, NL_m, NL_n, "pendulum")
        sys_model.InitSequence(NL_m1_0, NL_m2_0)
        
        [_, _, _, MSE_EKF_dB_avg_total_original, MSE_EKF_dB_avg_theta_original, MSE_EKF_dB_avg_omega_original, _, _, _, _, _] = \
            EKFTest(SysModel=sys_model, test_input=input_test, test_target=target_test, model_AE_conv=None,
                    matrix_data_flag=True, modelKnowledge=modelKnow, true_init_flag=true_init, random_init_flag=random_init)
        print(f"SAVING MSE_DB_TOTAL TEST SET : {MSE_EKF_dB_avg_total}")
        """
        os.makedirs(ROOT_PATH+f"Simulations_results/Pendulum/EKF/constant_nu_grid_new/{modelKnow}_observation_model/opt_wrt_{opt_wrt_to}/", exist_ok=True)
        """
        np.savez(ROOT_PATH+f"Simulations_results/Pendulum/EKF/no_grid/{modelKnow}_observation_model/v_{v}db_r2_{r2:.0e}_q2_{q2:.0e}.npz",
                  MSE_linear_arr_total=MSE_EKF_linear_arr_total.cpu(), MSE_linear_arr_theta=MSE_EKF_linear_arr_theta.cpu(),
                  MSE_linear_arr_omega=MSE_EKF_linear_arr_omega.cpu(), MSE_db_avg_total=MSE_EKF_dB_avg_total.cpu(),
                  MSE_db_avg_theta=MSE_EKF_dB_avg_theta.cpu(), MSE_db_avg_omega=MSE_EKF_dB_avg_omega.cpu(),
                  MSE_EKF_dB_std_total=MSE_EKF_dB_std_total.cpu(), MSE_EKF_dB_std_theta=MSE_EKF_dB_std_theta.cpu(),
                  MSE_EKF_dB_std_omega=MSE_EKF_dB_std_omega.cpu(), KG_array=KG_array.cpu(), results_plain=EKF_out.cpu(),
                  best_q2=best_q2, best_r2=best_r2, MSE_original_total=MSE_EKF_dB_avg_total_original.cpu(),
                  MSE_original_theta=MSE_EKF_dB_avg_theta_original.cpu(), MSE_original_omega=MSE_EKF_dB_avg_omega_original.cpu())
        """
        np.savez(ROOT_PATH+f"Simulations_results/Pendulum/EKF/constant_nu_grid_new/{modelKnow}_observation_model/opt_wrt_{opt_wrt_to}/v_{v}db_r2_{r2:.0e}_q2_{q2:.0e}.npz",
                  MSE_linear_arr_total=MSE_EKF_linear_arr_total.cpu(), MSE_linear_arr_theta=MSE_EKF_linear_arr_theta.cpu(),
                  MSE_linear_arr_omega=MSE_EKF_linear_arr_omega.cpu(), MSE_db_avg_total=MSE_EKF_dB_avg_total.cpu(),
                  MSE_db_avg_theta=MSE_EKF_dB_avg_theta.cpu(), MSE_db_avg_omega=MSE_EKF_dB_avg_omega.cpu(),
                  MSE_EKF_dB_std_total=MSE_EKF_dB_std_total.cpu(), MSE_EKF_dB_std_theta=MSE_EKF_dB_std_theta.cpu(),
                  MSE_EKF_dB_std_omega=MSE_EKF_dB_std_omega.cpu(), KG_array=KG_array.cpu(), results_plain=EKF_out.cpu(),
                  q2=best_q2, r2=best_r2)
        print("\n")

Streaming output truncated to the last 5000 lines.
Inference Time: 14.437374114990234
best_mSE: -27.520597457885742, MSE_total: -18.682979583740234, MSE_total:tensor(-18.6830)


Evaluate grid-value q2: 0.15 
EKF - MSE LOSS TOTAL: tensor(-24.5710) [dB]
EKF - MSE STD TOTAL: tensor(1.2769) [dB]
Inference Time: 14.53701663017273
best_mSE: -27.520597457885742, MSE_total: -24.571002960205078, MSE_total:tensor(-24.5710)


Evaluate grid-value q2: 0.001 
EKF - MSE LOSS TOTAL: tensor(-25.7546) [dB]
EKF - MSE STD TOTAL: tensor(2.4251) [dB]
Inference Time: 14.390556812286377
best_mSE: -27.520597457885742, MSE_total: -25.754638671875, MSE_total:tensor(-25.7546)


Evaluate grid-value q2: 0.0001 
EKF - MSE LOSS TOTAL: tensor(-23.8635) [dB]
EKF - MSE STD TOTAL: tensor(2.7303) [dB]
Inference Time: 14.531280279159546
best_mSE: -27.520597457885742, MSE_total: -23.863510131835938, MSE_total:tensor(-23.8635)


Evaluate grid-value q2: 0.2 
EKF - MSE LOSS TOTAL: tensor(-23.7802) [dB]
EKF - MSE STD TOTAL: ten

Exception ignored in: <function NpzFile.__del__ at 0x7f8394eb13b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py", line 224, in __del__
  File "/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py", line 219, in close
OSError: [Errno 107] Transport endpoint is not connected


EKF - MSE LOSS TOTAL: tensor(-3.7272) [dB]
EKF - MSE STD TOTAL: tensor(4.5892) [dB]
Inference Time: 15.461342811584473
best_mSE: 1000, MSE_total: -3.7271571159362793, MSE_total:tensor(-3.7272)


Evaluate grid-value q2: 20.0 
EKF - MSE LOSS TOTAL: tensor(-0.2640) [dB]
EKF - MSE STD TOTAL: tensor(4.0826) [dB]
Inference Time: 15.228539228439331
best_mSE: -3.7271571159362793, MSE_total: -0.2639935314655304, MSE_total:tensor(-0.2640)


Evaluate grid-value q2: 200.0 
EKF - MSE LOSS TOTAL: tensor(9.7224) [dB]
EKF - MSE STD TOTAL: tensor(3.5385) [dB]
Inference Time: 15.286059856414795
best_mSE: -3.7271571159362793, MSE_total: 9.722403526306152, MSE_total:tensor(9.7224)


Evaluate grid-value q2: 30.0 
EKF - MSE LOSS TOTAL: tensor(0.8653) [dB]
EKF - MSE STD TOTAL: tensor(4.1511) [dB]
Inference Time: 15.437550067901611
best_mSE: -3.7271571159362793, MSE_total: 0.8652811050415039, MSE_total:tensor(0.8653)


Evaluate grid-value q2: 0.2 
EKF - MSE LOSS TOTAL: tensor(-2.8473) [dB]
EKF - MSE STD TOTAL

In [ ]:
q2s = [0, -10, -20]
q2s = list(map(lambda x: 10**(x/10), q2s))
q2s

[1.0, 0.1, 0.01]